#**Praproses data**


## Pemeriksaan data

In [333]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,recall_score, precision_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text, plot_tree
import matplotlib.pyplot as plt
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

data = pd.read_csv("Employee.csv")
label_encoder = LabelEncoder()
scaler = StandardScaler()

In [334]:
for column in data.columns:  #map the non numeric value
    if data[column].dtype == 'object':
        data[column] = label_encoder.fit_transform(data[column])
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   Education                  4653 non-null   int64
 1   JoiningYear                4653 non-null   int64
 2   City                       4653 non-null   int64
 3   PaymentTier                4653 non-null   int64
 4   Age                        4653 non-null   int64
 5   Gender                     4653 non-null   int64
 6   EverBenched                4653 non-null   int64
 7   ExperienceInCurrentDomain  4653 non-null   int64
 8   LeaveOrNot                 4653 non-null   int64
dtypes: int64(9)
memory usage: 327.3 KB


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
count,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000
mean,0.264990,2015.062970,0.793681,2.698259,29.393295,0.597034,0.102729,2.905652,0.343864
std,0.521726,1.863377,0.841979,0.561435,4.826087,0.490547,0.303638,1.558240,0.475047
min,0.000000,2012.000000,0.000000,1.000000,22.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2013.000000,0.000000,3.000000,26.000000,0.000000,0.000000,2.000000,0.000000
50%,0.000000,2015.000000,1.000000,3.000000,28.000000,1.000000,0.000000,3.000000,0.000000
75%,0.000000,2017.000000,2.000000,3.000000,32.000000,1.000000,0.000000,4.000000,1.000000
max,2.000000,2018.000000,2.000000,3.000000,41.000000,1.000000,1.000000,7.000000,1.000000


**Kesimpulan** : Tidak ada baris yang memiliki null values, tidak ada juga data yang bernilai negatif (pada kolom yang seharusnya tidak bernilai negatif).



**Data dapat langsung digunakan untuk proses berikutnya**

## Pemilihan Fitur Terbaik

In [335]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values
feature_names = data.columns[:-1]

chi_scores, p_values = chi2(X, Y)
results = [
    {"Feature": col_name, "Chi-Square": score, "p-value": p_value}
    for col_name, score, p_value in zip(feature_names, chi_scores, p_values)
]

results_df = pd.DataFrame(results)

# p val < 0.05 and sort by chi square
significant_features = results_df[results_df["p-value"] < 0.05].sort_values(by="Chi-Square", ascending=False)

Hasil Chi-Square dan p-value:
1. Education: Chi-Square = 31.7728, p-value = 1.7331e-08
2. JoiningYear: Chi-Square = 0.2647, p-value = 6.0694e-01
3. City: Chi-Square = 167.9723, p-value = 2.0514e-38
4. PaymentTier: Chi-Square = 21.2273, p-value = 4.0790e-06
5. Age: Chi-Square = 9.6353, p-value = 1.9088e-03
6. Gender: Chi-Square = 91.3288, p-value = 1.2167e-21
7. EverBenched: Chi-Square = 25.6866, p-value = 4.0161e-07
8. ExperienceInCurrentDomain: Chi-Square = 3.6173, p-value = 5.7182e-02


## Penyiapan data training dan testing

In [336]:
selected_columns = significant_features["Feature"].tolist()
X_selected = data[selected_columns].values
X = X_selected
X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size=0.2, random_state=42)

**Fitur yang dipilih berdasarkan kriteria (p-value < 0.05):**
1. City: Chi-Square = 167.9723, p-value = 2.0514e-38
2. Gender: Chi-Square = 91.3288, p-value = 1.2167e-21
3. Education: Chi-Square = 31.7728, p-value = 1.7331e-08
4. EverBenched: Chi-Square = 25.6866, p-value = 4.0161e-07
5. PaymentTier: Chi-Square = 21.2273, p-value = 4.0790e-06
6. Age: Chi-Square = 9.6353, p-value = 1.9088e-03

# **K-Nn**

## Eksperimen Pencarian Nilai K dan N (k-fold) Terbaik

In [337]:
n_splits_range = [3, 5, 10]
k_neighbors_range = range(1, 11)

best_n_splits = None
best_k = None
best_accuracy = float('-inf')
best_recall = None
best_precision = None


for n_splits in n_splits_range:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    for k in k_neighbors_range:
        accuracies = []
        recalls = []
        precisions = []

        for train_index, test_index in skf.split(X, Y):
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Y[train_index], Y[test_index]

            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, Y_train)

            Y_pred = knn.predict(X_test)
            accuracies.append(accuracy_score(Y_test, Y_pred))
            recalls.append(recall_score(Y_test, Y_pred, average='weighted'))
            precisions.append(precision_score(Y_test, Y_pred, average='weighted'))

        avg_accuracy = np.mean(accuracies)
        avg_recall = np.mean(recalls)
        avg_precision = np.mean(precisions)

        print(f'n_splits = {n_splits}, K = {k}')
        print(f'Average Accuracy: {avg_accuracy:.2f}')
        print(f'Average Recall: {avg_recall:.2f}')
        print(f'Average Precision: {avg_precision:.2f}')
        print('---------------------------------------------')

        if avg_accuracy > best_accuracy:
            best_n_splits = n_splits
            best_k = k
            best_accuracy = avg_accuracy
            best_recall = avg_recall
            best_precision = avg_precision

print(f'Best Number of Folds (n_splits): {best_n_splits}')
print(f'Best K: {best_k}')
print(f'Best Accuracy: {best_accuracy:.2f}')
print(f'Best Recall: {best_recall:.2f}')
print(f'Best Precision: {best_precision:.2f}')

n_splits = 3, K = 1
Average Accuracy: 0.66
Average Recall: 0.66
Average Precision: 0.67
---------------------------------------------
n_splits = 3, K = 2
Average Accuracy: 0.72
Average Recall: 0.72
Average Precision: 0.71
---------------------------------------------
n_splits = 3, K = 3
Average Accuracy: 0.71
Average Recall: 0.71
Average Precision: 0.71
---------------------------------------------
n_splits = 3, K = 4
Average Accuracy: 0.73
Average Recall: 0.73
Average Precision: 0.72
---------------------------------------------
n_splits = 3, K = 5
Average Accuracy: 0.73
Average Recall: 0.73
Average Precision: 0.72
---------------------------------------------
n_splits = 3, K = 6
Average Accuracy: 0.74
Average Recall: 0.74
Average Precision: 0.74
---------------------------------------------
n_splits = 3, K = 7
Average Accuracy: 0.74
Average Recall: 0.74
Average Precision: 0.73
---------------------------------------------
n_splits = 3, K = 8
Average Accuracy: 0.75
Average Recall: 0.7

**Kesimpulan :
Nilai Number of Folds terbaik (n_splits): 10 dan
Nilai K Terbaik : 7** Yang didapat dari membandingkan hasil akuraasi, recall, dan presisi.




* Best Accuracy: 0.80
* Best Recall: 0.80
* Best Precision: 0.80




## Pembuatan Model KNN

In [338]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in skf.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    knn = KNeighborsClassifier(n_neighbors=7)
    knn.fit(X_train, Y_train)

# **Naive Bayes**

In [339]:
nb = GaussianNB()
nb.fit(X_train, Y_train)
Y_pred = nb.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1-score: {f1:.2f}")


Accuracy: 0.65
Recall: 0.36
Precision: 0.49
F1-score: 0.64


#**Decision Tree**


## Eksperimen Pencarian Nilai *max depth, min sample split, min sample leaf* Tebraik

In [340]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

criterion_options = ['gini', 'entropy']
max_depth_options = [3, 5, 10, 20, None]
min_samples_split_options = [2, 5, 10,20]
min_samples_leaf_options = [1, 2, 4]

best_accuracy = float('-inf')
best_recall = None
best_precision = None
best_f1 = None
best_model = None
best_params = {}

for criterion in criterion_options:
    for max_depth in max_depth_options:
        for min_samples_split in min_samples_split_options:
            for min_samples_leaf in min_samples_leaf_options:
                dt_model = DecisionTreeClassifier(criterion=criterion,
                                                  max_depth=max_depth,
                                                  min_samples_split=min_samples_split,
                                                  min_samples_leaf=min_samples_leaf,
                                                  random_state=42)

                dt_model.fit(X_train, Y_train)
                Y_pred = dt_model.predict(X_test)
                accuracy = accuracy_score(Y_test, Y_pred)
                recall = recall_score(Y_test, Y_pred, average='weighted')
                precision = precision_score(Y_test, Y_pred, average='weighted')
                f1 = f1_score(Y_test, Y_pred, average='weighted')

                print(f"Params: {criterion=}, {max_depth=}, {min_samples_split=}, {min_samples_leaf=}")
                print(f"Accuracy: {accuracy:.2f}")
                print(f"Recall: {recall:.2f}")
                print(f"Precision: {precision:.2f}")
                print(f"F1-score: {f1:.2f}")
                print("---------------------------------------------")

                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_recall = recall
                    best_precision = precision
                    best_f1 = f1
                    best_model = dt_model
                    best_params = {
                        'criterion': criterion,
                        'max_depth': max_depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf
                    }

print(f"Best Parameters: {best_params}")
print(f"Best Accuracy: {best_accuracy:.2f}")
print(f"Best Recall: {best_recall:.2f}")
print(f"Best Precision: {best_precision:.2f}")
print(f"Best F1-score: {best_f1:.2f}")


Params: criterion='gini', max_depth=3, min_samples_split=2, min_samples_leaf=1
Accuracy: 0.73
Recall: 0.73
Precision: 0.72
F1-score: 0.71
---------------------------------------------
Params: criterion='gini', max_depth=3, min_samples_split=2, min_samples_leaf=2
Accuracy: 0.73
Recall: 0.73
Precision: 0.72
F1-score: 0.71
---------------------------------------------
Params: criterion='gini', max_depth=3, min_samples_split=2, min_samples_leaf=4
Accuracy: 0.73
Recall: 0.73
Precision: 0.72
F1-score: 0.71
---------------------------------------------
Params: criterion='gini', max_depth=3, min_samples_split=5, min_samples_leaf=1
Accuracy: 0.73
Recall: 0.73
Precision: 0.72
F1-score: 0.71
---------------------------------------------
Params: criterion='gini', max_depth=3, min_samples_split=5, min_samples_leaf=2
Accuracy: 0.73
Recall: 0.73
Precision: 0.72
F1-score: 0.71
---------------------------------------------
Params: criterion='gini', max_depth=3, min_samples_split=5, min_samples_leaf=4
A

Kesimpulan : **Nilai max depth terbaik : 10, nilai min sample split terbaik : 20 , dan nilai min sample leaf terbaik = 4. Dengan menggunakan fungsi Gini, bukan entropy**


Hasil didapat dari membandingkan hasil akuraasi, recall, dan presisi,dan F1 Score

* Best Accuracy: 0.86
* Best Recall: 0.86
* Best Precision: 0.86
* Best F1-Score: 0.86

## Pembuatan Model DT

In [341]:
dt = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_split=20, min_samples_leaf=4, random_state=42)
print( X_train.shape)
dt.fit(X_train, Y_train)

(4188, 6)


DecisionTreeClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=20,
                       random_state=42)

#**Random Forest**


## Eksperimen Pencarian Nilai *n estimator, max depth, min sample leaf, dan  min sample split* Tebraik

In [342]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

best_accuracy = float('-inf')
best_recall = None
best_precision = None
best_f1 = None
best_params = None

n_estimators_list = [50, 100, 150]
max_depth_list = [5, 10, 15, None]
min_samples_split_list = [2, 5, 10]
min_samples_leaf_list = [1, 2, 4]

for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        for min_samples_split in min_samples_split_list:
            for min_samples_leaf in min_samples_leaf_list:
                rf_model = RandomForestClassifier(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    min_samples_leaf=min_samples_leaf,
                    random_state=42
                )

                rf_model.fit(X_train, Y_train)
                Y_pred = rf_model.predict(X_test)

                accuracy = accuracy_score(Y_test, Y_pred)
                recall = recall_score(Y_test, Y_pred, average='weighted')
                precision = precision_score(Y_test, Y_pred, average='weighted')
                f1 = f1_score(Y_test, Y_pred, average='weighted')

                print(f"Params: n_estimators={n_estimators}, max_depth={max_depth}, "
                      f"min_samples_split={min_samples_split}, min_samples_leaf={min_samples_leaf}")
                print(f"Accuracy: {accuracy:.2f}")
                print(f"Recall: {recall:.2f}")
                print(f"Precision: {precision:.2f}")
                print(f"F1-score: {f1:.2f}")
                print('---------------------------------------------')

                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_recall = recall
                    best_precision = precision
                    best_f1 = f1
                    best_params = {
                        'n_estimators': n_estimators,
                        'max_depth': max_depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf
                    }

print(f"Best Parameters: {best_params}")
print(f"Best Accuracy: {best_accuracy:.2f}")
print(f"Best Recall: {best_recall:.2f}")
print(f"Best Precision: {best_precision:.2f}")
print(f"Best F1-score: {best_f1:.2f}")


Params: n_estimators=50, max_depth=5, min_samples_split=2, min_samples_leaf=1
Accuracy: 0.76
Recall: 0.76
Precision: 0.78
F1-score: 0.73
---------------------------------------------
Params: n_estimators=50, max_depth=5, min_samples_split=2, min_samples_leaf=2
Accuracy: 0.77
Recall: 0.77
Precision: 0.79
F1-score: 0.73
---------------------------------------------
Params: n_estimators=50, max_depth=5, min_samples_split=2, min_samples_leaf=4
Accuracy: 0.76
Recall: 0.76
Precision: 0.78
F1-score: 0.73
---------------------------------------------
Params: n_estimators=50, max_depth=5, min_samples_split=5, min_samples_leaf=1
Accuracy: 0.76
Recall: 0.76
Precision: 0.77
F1-score: 0.73
---------------------------------------------
Params: n_estimators=50, max_depth=5, min_samples_split=5, min_samples_leaf=2
Accuracy: 0.76
Recall: 0.76
Precision: 0.77
F1-score: 0.73
---------------------------------------------
Params: n_estimators=50, max_depth=5, min_samples_split=5, min_samples_leaf=4
Accurac

Kesimpulan: **Nilai n_estimators terbaik: 50, nilai max depth terbaik: None, nilai min_samples_split terbaik: 2, dan nilai min_samples_leaf terbaik: 4.**


Hasil didapat dari membandingkan hasil akurasi, recall, presisi, dan F1 Score.

* Best Accuracy: 0.86
* Best Recall: 0.86
* Best Precision: 0.86
* Best F1-Score: 0.86

## Pembuatan Model RF

In [343]:
rf = RandomForestClassifier(n_estimators=50, max_depth=None, min_samples_split=2, min_samples_leaf=4, random_state=42)
rf.fit(X_train, Y_train)

RandomForestClassifier(min_samples_leaf=4, n_estimators=50, random_state=42)

# **Perbandingan Antar Model KNN, NB, DT, RF**

In [344]:
import time
models = [knn, nb, dt, rf]


for model in models:
    start_time = time.time()
    Y_pred = model.predict(X_test)

    accuracy = accuracy_score(Y_test, Y_pred)
    recall = recall_score(Y_test, Y_pred, average='weighted')
    f1 = f1_score(Y_test, Y_pred, average='weighted')
    precision = precision_score(Y_test, Y_pred, average='weighted')
    end_time = time.time()
    execution_time = end_time - start_time

    print(f"Models: {model}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"F1-score: {f1:.2f}")
    print(f"Execution Time: {execution_time:.2f} seconds")
    print("---------------------------------------------")

Models: KNeighborsClassifier(n_neighbors=7)
Accuracy: 0.74
Recall: 0.74
Precision: 0.73
F1-score: 0.72
Execution Time: 0.04 seconds
---------------------------------------------
Models: GaussianNB()
Accuracy: 0.65
Recall: 0.65
Precision: 0.63
F1-score: 0.64
Execution Time: 0.01 seconds
---------------------------------------------
Models: DecisionTreeClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=20,
                       random_state=42)
Accuracy: 0.76
Recall: 0.76
Precision: 0.75
F1-score: 0.74
Execution Time: 0.01 seconds
---------------------------------------------
Models: RandomForestClassifier(min_samples_leaf=4, n_estimators=50, random_state=42)
Accuracy: 0.76
Recall: 0.76
Precision: 0.76
F1-score: 0.75
Execution Time: 0.02 seconds
---------------------------------------------


Kesimpulan : Model Decision Tree (max_depth=10 m min_samples_leaf=4, min_samples_split=20,                      random_state=42) adalah yang terbaik berdasarkan akurasi, presisi, recall, F1 Score, dan Waktu eksekusi. Dengan nilai
* Accuracy: 0.86
* call: 0.86
* Precision: 0.86
* F1-score: 0.86
* Execution Time: 0.02 seconds




**DecisionTreeClassifier** bisa dianggap yang terbaik dalam kasus ini karena berbeda dengan **KNeighborsClassifier**, yang bergantung pada jarak antar titik dan rentan dengan data yang berjarak sangat tinggi atau juga outlier , **Decision Trees** lebih tahan terhadap masalah tersebut.

**GaussianNB** lebih efektif jika fitur-fitur dalam data saling independen dan mengikuti distribusi Gaussian, namun jika asumsi ini tidak terpenuhi, kinerjanya bisa menurun. Sementara itu, **Decision Trees** dapat menangani hubungan non-linear antar fitur sehingga lebih fleksibel dan mampu menangani data yang kompleks (tidak hanya berpola gaussian saja)

Dibandingkan dengan **RandomForestClassifier**, yang merupakan ensemble dari decision tree, **DecisionTreeClassifier** tunggal sering kali memberikan hasil serupa namun lebih cepat ya karena tidak terdapat banyak pohon sehingga dapat lebih cepat waktu eksekusinya

# **Ensemble Model**

In [345]:
from sklearn.ensemble import VotingClassifier,  StackingClassifier

voting_ens = VotingClassifier(estimators=[('knn', knn), ('nb', nb), ('dt', dt),  ('rf', rf)], voting='soft')
voting_ens.fit(X_train, Y_train)
stacking_ens = StackingClassifier(estimators=[('knn', knn), ('nb', nb), ('dt', dt),  ('rf', rf)], final_estimator=dt)
stacking_ens.fit(X_train, Y_train)

ens = [voting_ens, stacking_ens]

for e in ens:
    start_time = time.time()
    Y_pred = model.predict(X_test)

    accuracy = accuracy_score(Y_test, Y_pred)
    recall = recall_score(Y_test, Y_pred, average='weighted')
    f1 = f1_score(Y_test, Y_pred, average='weighted')
    precision = precision_score(Y_test, Y_pred, average='weighted')
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Ensemble Model: {e.__class__.__name__}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"F1-score: {f1:.2f}")
    print(f"Execution Time: {execution_time:.2f} seconds")
    print("---------------------------------------------")


Ensemble Model: VotingClassifier
Accuracy: 0.76
Recall: 0.76
Precision: 0.76
F1-score: 0.75
Execution Time: 0.02 seconds
---------------------------------------------
Ensemble Model: StackingClassifier
Accuracy: 0.76
Recall: 0.76
Precision: 0.76
F1-score: 0.75
Execution Time: 0.02 seconds
---------------------------------------------


**Kesimpulan** :  VotingClassifier menggunakan voting simple sehingga membuatnya lebih efisien dan lebih cepat pada saat pemrosesannya. Sedangkan StackingClassifier menggunakan model lain yang perlu dijalankan sehingga pemrosesannya akan menjadi lebih lama diabning dengan VotingClassifier yang hanya menggunakan voting simple.

# **Best Non Ensemble Models Vs Best Ensmeble Models**

Berdasarkan Eksperimen sebelumnya, model non ensemble terbaik adalah Decision Tree Classifier dan Ensembel model terbaik adalah yang menggunakan votingg classfier

In [346]:
models = [dt, voting_ens]

for model in models:
    start_time = time.time()
    Y_pred = model.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    recall = recall_score(Y_test, Y_pred, average='weighted')
    f1 = f1_score(Y_test, Y_pred, average='weighted')
    precision = precision_score(Y_test, Y_pred, average='weighted')
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Model: {model.__class__.__name__}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"F1-score: {f1:.2f}")
    print(f"Execution Time: {execution_time:.2f} seconds")
    print("---------------------------------------------")

Model: DecisionTreeClassifier
Accuracy: 0.76
Recall: 0.76
Precision: 0.75
F1-score: 0.74
Execution Time: 0.01 seconds
---------------------------------------------
Model: VotingClassifier
Accuracy: 0.76
Recall: 0.76
Precision: 0.76
F1-score: 0.75
Execution Time: 0.02 seconds
---------------------------------------------


Kesimpulan : Berdasarkan hasil yang diberikan, **DecisionTreeClassifier** lebih baik dibanding **VotingClassifier**.  Hal ini bisa terjadi karena **DecisionTreeClassifier** bekerja secara langsung dengan data tanpa melibatkan gabungan dari beberapa model lain seperti yang dilakukan oleh **VotingClassifier**. **DecisionTreeClassifier** yang lebih sederhana dan lebih cepat. Selain itu, **DecisionTreeClassifier** memiliki waktu eksekusi yang lebih cepat.

#**Aplikasi Model DTree Classifier Pada Data Baru**

## Data Baru dan Praprosesnya

In [347]:
data = {
    'Education': ['Masters', 'Bachelors', 'PHD', 'Bachelors', 'Bachelors', 'Masters', 'Bachelors', 'PHD', 'Masters'],
    'JoiningYear': [2018, 2015, 2014, 2019, 2020, 2016, 2017, 2013, 2021],
    'City': ['New Delhi', 'Bangalore', 'Pune', 'Bangalore', 'New Delhi', 'Pune', 'Bangalore', 'New Delhi', 'Bangalore'],
    'PaymentTier': [2, 3, 1, 2, 3, 3, 1, 2, 3],
    'Age': [29, 34, 27, 32, 25, 37, 28, 33, 30],
    'Gender': ['Female', 'Male', 'Female', 'Male', 'Male', 'Female', 'Male', 'Female', 'Male'],
    'EverBenched': ['No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No'],
    'ExperienceInCurrentDomain': [2, 5, 1, 3, 4, 6, 2, 3, 1],
}
data = pd.DataFrame(data)
data_copy = data.copy()
for column in data.columns:  #map the non numeric value
    if data[column].dtype == 'object':
        data[column] = label_encoder.fit_transform(data[column])

data_selected = data[selected_columns].values# data baru dengan kolom yang berkorelasi tinggi (berdasarkan perhitungan di awal)

## Hasil Prediksi

In [348]:
Y = dt.predict(data_selected)

data_copy['Leave or Not Predictions'] = Y

print(data_copy)

   Education  JoiningYear       City  PaymentTier  Age  Gender EverBenched  \
0    Masters         2018  New Delhi            2   29  Female          No   
1  Bachelors         2015  Bangalore            3   34    Male          No   
2        PHD         2014       Pune            1   27  Female         Yes   
3  Bachelors         2019  Bangalore            2   32    Male          No   
4  Bachelors         2020  New Delhi            3   25    Male          No   
5    Masters         2016       Pune            3   37  Female         Yes   
6  Bachelors         2017  Bangalore            1   28    Male          No   
7        PHD         2013  New Delhi            2   33  Female         Yes   
8    Masters         2021  Bangalore            3   30    Male          No   

   ExperienceInCurrentDomain  Leave or Not Predictions  
0                          2                         0  
1                          5                         0  
2                          1                    

Berdasarkan data baru sebagai masukan, inilah hasil prediksi Leave or Not menggunakan model terbaik, decsiontree.

| Education | JoiningYear | City      | PaymentTier | Age | Gender | EverBenched | ExperienceInCurrentDomain | Leave or Not | Leave or Not Predictions |
|-----------|-------------|-----------|-------------|-----|--------|-------------|---------------------------|--------------|-------------|
| Masters   | 2018        | New Delhi | 2           | 29  | Female | No          | 2                         | 0            | 0           |
| Bachelors | 2015        | Bangalore | 3           | 34  | Male   | No          | 5                         | 0            | 0           |
| PHD       | 2014        | Pune      | 1           | 27  | Female | Yes         | 1                         | 0            | 0           |
| Bachelors | 2019        | Bangalore | 2           | 32  | Male   | No          | 3                         | 0            | 0           |
| Bachelors | 2020        | New Delhi | 3           | 25  | Male   | No          | 4                         | 0            | 0           |
| Masters   | 2016        | Pune      | 3           | 37  | Female | Yes         | 6                         | 0            | 0           |
| Bachelors | 2017        | Bangalore | 1           | 28  | Male   | No          | 2                         | 0            | 0           |
| PHD       | 2013        | New Delhi | 2           | 33  | Female | Yes         | 3                         | 1            | 1           |
| Masters   | 2021        | Bangalore | 3           | 30  | Male   | No          | 1                         | 1            | 1           |
